In [7]:
from ssi_fc_data import fc_md_client , model
import config
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
client = fc_md_client.MarketDataClient(config)
# data = client.intraday_ohlc(config, model.intraday_ohlc('vn30', '01/10/2020', '31/10/2020', 1, 9999, True, 1))['data']
# data = data['data']
data = pd.DataFrame(client.intraday_ohlc(config, model.intraday_ohlc('vn30', '01/10/2020', '31/10/2020', 1, 9999, True, 1))['data'])
data 

,Symbol,Value,TradingDate,Time,Open,High,Low,Close,Volume
0,VN30,863.54,01/10/2020,09:15:56,860.25,863.54,860.25,863.54,1433440
1,VN30,864.16,01/10/2020,09:16:56,863.64,864.16,863.61,864.16,554880
2,VN30,864.70,01/10/2020,09:17:56,864.08,864.70,863.97,864.70,579470
3,VN30,865.42,01/10/2020,09:18:56,864.64,865.42,864.38,865.42,903290
4,VN30,865.07,01/10/2020,09:19:56,865.31,865.54,864.81,865.07,800260
...,...,...,...,...,...,...,...,...,...
5009,VN30,885.69,30/10/2020,14:27:58,885.78,885.79,885,885.69,906080
5010,VN30,885.20,30/10/2020,14:28:58,885.53,886.05,885.20,885.20,661090
5011,VN30,885.51,30/10/2020,14:29:58,884.96,885.51,884.86,885.51,723390
5012,VN30,884.93,30/10/2020,14:30:18,885.29,885.29,884.90,884.93,104060


In [8]:
def get_vn30_data(start_date, end_date):
    """
    Fetch VN30 intraday OHLC data for the period between start_date and end_date.
    The start_date and end_date should be strings in "dd/mm/yyyy" format.
    
    The function splits the overall period by month (using fixed rules for month-end days,
    e.g. February is forced to 28 days, and April, June, September, and November to 30 days)
    and then concatenates the data from each segment.
    """

    # Instantiate the client (assumes config and model are available in scope)
    client = fc_md_client.MarketDataClient(config)
    
    # Convert the input strings to datetime objects.
    start_dt = datetime.strptime(start_date, "%d/%m/%Y")
    end_dt = datetime.strptime(end_date, "%d/%m/%Y")
    
    data_list = []
    
    # Start iteration from the month/year of start_dt to the month/year of end_dt.
    current_year = start_dt.year
    current_month = start_dt.month

    while (current_year < end_dt.year) or (current_year == end_dt.year and current_month <= end_dt.month):
        # Determine the starting day for this month:
        if current_year == start_dt.year and current_month == start_dt.month:
            day_start = start_dt.day
        else:
            day_start = 1

        # By default, set the month's end day based on the original logic:
        # February -> 28, April/June/Sept/Nov -> 30, all others -> 31.
        if current_month == 2:
            max_day = 28
        elif current_month in [4, 6, 9, 11]:
            max_day = 30
        else:
            max_day = 31

        # If this is the ending month, adjust the end day to the provided end date.
        if current_year == end_dt.year and current_month == end_dt.month:
            day_end = end_dt.day
        else:
            day_end = max_day

        # Build the date strings in "dd/mm/yyyy" format.
        month_str = str(current_month).zfill(2)
        seg_start_date = f"{str(day_start).zfill(2)}/{month_str}/{current_year}"
        seg_end_date = f"{str(day_end).zfill(2)}/{month_str}/{current_year}"
        
        # Fetch data for this monthly segment.
        result = client.intraday_ohlc(
            config,
            model.intraday_ohlc('vn30', seg_start_date, seg_end_date, 1, 9999, True, 1)
        )
        df = pd.DataFrame(result['data'])
        data_list.append(df)
        
        # Move to the next month.
        if current_month == 12:
            current_month = 1
            current_year += 1
        else:
            current_month += 1

    # Concatenate all monthly DataFrames into a single DataFrame.
    data = pd.concat(data_list, ignore_index=True)
    return data

In [9]:
data = get_vn30_data('01/10/2020', '31/10/2021')
data

,Symbol,Value,TradingDate,Time,Open,High,Low,Close,Volume
0,VN30,863.54,01/10/2020,09:15:56,860.25,863.54,860.25,863.54,1433440
1,VN30,864.16,01/10/2020,09:16:56,863.64,864.16,863.61,864.16,554880
2,VN30,864.70,01/10/2020,09:17:56,864.08,864.70,863.97,864.70,579470
3,VN30,865.42,01/10/2020,09:18:56,864.64,865.42,864.38,865.42,903290
4,VN30,865.07,01/10/2020,09:19:56,865.31,865.54,864.81,865.07,800260
...,...,...,...,...,...,...,...,...,...
61011,VN30,1532.04,29/10/2021,14:26:55,1532.20,1532.69,1531.62,1532.04,1095400
61012,VN30,1532.02,29/10/2021,14:27:55,1531.99,1532.87,1531.62,1532.02,796700
61013,VN30,1530.73,29/10/2021,14:28:56,1532.12,1532.51,1530.73,1530.73,1311700
61014,VN30,1531.62,29/10/2021,14:29:41,1530.82,1531.62,1530.82,1531.62,586800


In [ ]:
# data_list = []  # List to store DataFrames before concatenation

# for i in range(1, 13):
#     month = f'{i:02d}'  # Format month as '01', '02', ..., '12'
    
#     if i == 2:
#         end_date = '28/' + month + '/2023'
#     # Fetch data
#     df = pd.DataFrame(client.intraday_ohlc(config, model.intraday_ohlc('vn30', f'01/{month}/2023', f'31/{month}/2023', 1, 9999, True, 1))['data'])
    
#     data_list.append(df)  # Append each DataFrame to the list

# # Concatenate all DataFrames into a single DataFrame
# data = pd.DataFrame(data_list, ignore_index=True)
# data

KeyError: 'data'

In [5]:
# Getting VN30 data from 01/01/2024 to 31/12/2024 by looping through each month in the string format 01/%s/2024
#if month < 10, add 0 before month
data_list = [] # List to store DataFrames before concatenation

for i in range(1, 13):    
    if i < 10:
        month = '0' + str(i)
    else:
        month = str(i)

    # month = '01'
    start_date = '01/' + month + '/2024'
    end_date = '31/' + month + '/2024'
    # Check if the month is February
    print(end_date)
    if i == 2:
        end_date = '28/' + month + '/2024'
    
    # Check if the month has 30 days
    if i in [4, 6, 9, 11]:
        end_date = '30/' + month + '/2024'
    # Fetch data
    df = pd.DataFrame(client.intraday_ohlc(config, model.intraday_ohlc('vn30', start_date, end_date, 1, 9999, True, 1))['data'])
    
    data_list.append(df)  # Append each DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
data = pd.concat(data_list, ignore_index=True)

data
data


31/01/2024
31/02/2024
31/03/2024
31/04/2024
31/05/2024
31/06/2024
31/07/2024
31/08/2024
31/09/2024
31/10/2024
31/11/2024
31/12/2024


,Symbol,Value,TradingDate,Time,Open,High,Low,Close,Volume
0,VN30,1137.97,02/01/2024,09:15:56,1135.49,1138.19,1135.49,1137.97,919400
1,VN30,1137.95,02/01/2024,09:16:56,1137.79,1138.98,1137.74,1137.95,883300
2,VN30,1138.40,02/01/2024,09:17:56,1137.86,1138.74,1137.75,1138.40,745900
3,VN30,1138.16,02/01/2024,09:18:56,1138.17,1138.81,1138.05,1138.16,625800
4,VN30,1138.18,02/01/2024,09:19:56,1138.10,1138.59,1137.99,1138.18,811100
...,...,...,...,...,...,...,...,...,...
56748,VN30,1342.84,31/12/2024,14:27:59,1341.78,1343.28,1341.78,1342.84,822100
56749,VN30,1342.93,31/12/2024,14:28:59,1343.06,1343.42,1342.74,1342.93,935600
56750,VN30,1343.53,31/12/2024,14:29:59,1343.06,1343.53,1342.72,1343.53,1594300
56751,VN30,1343.62,31/12/2024,14:30:04,1343.62,1343.62,1343.62,1343.62,49800


In [6]:
data.to_csv('vn30_2024.csv', index=False)

In [ ]:
data_list = []